In [2758]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, decimal
import requests
dec = decimal.Decimal
import hvplot.pandas, holoviews as hv
import json
import streamlit as st

def get_crypto_price(symbol, exchange, days):
    api_key = 'YOUR API KEY'
    api_url = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym={symbol}&tsym={exchange}&limit={days}&api_key={api_key}'
    raw = requests.get(api_url).json()
    df = pd.DataFrame(raw['Data']['Data'])[['time', 'close']].set_index('time')
    df.index = pd.to_datetime(df.index, unit = 's')
    return df
FearGreedUrl = requests.get('https://api.alternative.me/fng/?limit=2').json()

In [2759]:
from pathlib import Path
csvpath1 = Path("./full_moon.csv")
lunar_eclipse = Path("./lunar_eclipse.csv")
solar_eclipse = Path("./solar_eclipse.csv")
mercury_retro = Path("./mercury_retrograde.csv")

# Retrieve BTC API

In [2760]:
btc = get_crypto_price('BTC', 'USD', 1825)
Price = btc['Price'] = btc['close']
SMA_9 = btc['9SMA'] = btc['close'].rolling(9).mean()
SMA_44 = btc['44SMA'] = btc['close'].rolling(44).mean()
SMA_117 = btc['117SMA'] = btc['close'].rolling(117).mean()
EMA_117 = btc['117EMA'] = btc['close'].ewm(117).mean()
btc = btc.reset_index()
btc = btc.rename(columns={'time':'dtime'})
btc.set_index(['dtime'], inplace=True)
btc_api = btc.drop(['close'], axis=1)
btc_api

,Price,9SMA,44SMA,117SMA,117EMA
dtime,,,,,
2017-06-29,2558.37,NaN,NaN,NaN,2558.370000
2017-06-30,2480.61,NaN,NaN,NaN,2519.324553
2017-07-01,2424.61,NaN,NaN,NaN,2487.483962
2017-07-02,2536.46,NaN,NaN,NaN,2499.884720
2017-07-03,2572.47,NaN,NaN,NaN,2514.649922
...,...,...,...,...,...
2022-06-24,21219.46,20426.332222,27339.663864,35693.161538,38113.986924
2022-06-25,21474.19,20548.703333,27170.538182,35507.560940,37972.971670
2022-06-26,21031.85,20615.324444,26983.879545,35307.589829,37829.402817


### Plot BTC Price over Time

In [2761]:
BTC = btc_api.hvplot.line(
    x = 'dtime',
    y = 'Price',
    xlabel = 'Date',
    ylabel = 'Price',
    title = 'BTC Price',
    legend = True
)
BTC

:Curve   [dtime]   (Price)

# Clean Lunar Data


In [2762]:
#Create a Full Moon Dataframe
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True)
full_moon.reset_index(inplace=True)
full_moon = full_moon.rename(columns = {' Date':'Full_Moon'})
full_moon.drop([' Time'], axis=1, inplace=True)

In [2763]:
# Create a Moon Data table with the BTC Price data for each Full and New Moon date

full_moon = full_moon.join(btc['Price'], on = 'Full_Moon', how = 'inner')
full_moon['Full Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price'])

full_moon.tail()

,Full_Moon,Full Moon Price
53,2022-02-16,43896.34
54,2022-03-18,41790.97
55,2022-04-16,40392.55
56,2022-05-16,29838.50
57,2022-06-14,22118.37


In [2764]:
#Create a New Moon df
new_moon = pd.DataFrame(full_moon['Full_Moon'] + timedelta(days=15))
new_moon.rename(columns={'Full_Moon':'New_Moon'}, inplace=True)
new_moon.drop(new_moon.index[-1], inplace=True)
new_moon = new_moon.rename(columns = {'Full_Moon':'New_Moon'})
new_moon = new_moon.join(btc['Price'], on = 'New_Moon', how = 'inner')
new_moon['New Moon Price'] = new_moon['Price']
new_moon = new_moon.drop(columns=['Price'])

new_moon.tail()

,New_Moon,New Moon Price
52,2022-02-02,36918.19
53,2022-03-03,42474.37
54,2022-04-02,45822.28
55,2022-05-01,38480.53
56,2022-05-31,31782.16


In [2765]:
moon_data = pd.concat([new_moon,full_moon], axis=1, join='inner')
# moon_data

In [2766]:
lunar_eclipse_df = pd.read_csv(lunar_eclipse, parse_dates=True, infer_datetime_format=True)
lunar_eclipse_df.rename(columns={'Date':'lunar eclipse'}, inplace=True)
lunar_eclipse_df['lunar eclipse'] = pd.to_datetime(lunar_eclipse_df['lunar eclipse'])

lunar_eclipse_df = lunar_eclipse_df.join(btc['Price'], on = 'lunar eclipse', how = 'inner')

In [2767]:
solar_eclipse_df = pd.read_csv(solar_eclipse, parse_dates=True, infer_datetime_format=True)
solar_eclipse_df.rename(columns={'Date':'solar eclipse'}, inplace=True)
solar_eclipse_df['solar eclipse'] = pd.to_datetime(solar_eclipse_df['solar eclipse'])
solar_eclipse_df.drop(['Unnamed: 1'], axis = 1, inplace=True)

solar_eclipse_df = solar_eclipse_df.join(btc['Price'], on = 'solar eclipse', how = 'inner')

In [2768]:
mercury_data = pd.read_csv(mercury_retro, parse_dates=True, infer_datetime_format=True)
mercury_data['Start'] = pd.to_datetime(mercury_data['Start'])
mercury_data['End'] = pd.to_datetime(mercury_data['End'])
mercury_data['Term'] = mercury_data['End'] - mercury_data['Start']
# mercury_data = pd.date_range(start=mercury_data['Start'], end=mercury_data['End'])
start = mercury_data['Start']
start_hv = start.hvplot.scatter()
end = mercury_data['End']
end_hv = end.hvplot.scatter()

mercury_data

# start_hv*end_hv

,Start,End,Term
0,2017-12-03,2017-12-22,19 days
1,2018-03-22,2018-04-15,24 days
2,2018-07-26,2018-08-18,23 days
3,2018-11-16,2018-12-06,20 days
4,2019-03-05,2019-03-28,23 days
5,2019-07-07,2019-07-31,24 days
6,2019-10-31,2019-11-20,20 days
7,2020-02-16,2020-03-09,22 days
8,2020-06-17,2020-07-12,25 days
9,2020-10-13,2020-11-03,21 days


# Plot Lunar Data over BTC Data

In [2769]:
# create hvplot figures to then overaly 
glyph_1 = btc['Price'].hvplot.line( 
    'dtime', 'Price',
    color='#e7e7e7',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_2 = full_moon.hvplot.scatter(
    x = 'Full_Moon',
    y = 'Full Moon Price',
    color='#ffcd33',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_3 = new_moon.hvplot.scatter(
    x = 'New_Moon',
    y = 'New Moon Price',
    color='#ff6533',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_4 = btc['9SMA'].hvplot.line(
    'dtime', '9SMA',
    color='#70eac4',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_9 = btc['44SMA'].hvplot.line(
    'dtime', '44SMA',
    color='pink',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_5 = btc['117SMA'].hvplot.line(
    'dtime', '117SMA',
    color='#55d24a',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_6 = btc['117EMA'].hvplot.line(
    'dtime', '117EMA',
    color='#22a91a',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_7 = lunar_eclipse_df.hvplot.scatter(
    x = 'lunar eclipse',
    y = 'Price',
    color='#505050',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_8 = solar_eclipse_df.hvplot.scatter(
    x = 'solar eclipse',
    y = 'Price',
    color='#000000',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

btc_time = glyph_1*glyph_2*glyph_3*glyph_4*glyph_5*glyph_6*glyph_9*glyph_7*glyph_8

## Prepare Lunar and BTC Data for Merge

In [2770]:
btc = get_crypto_price('BTC', 'USD', 1825)
Price = btc['Price'] = btc['close']
SMA_8 = btc['8SMA'] = btc['close'].rolling(8).mean()
SMA_55 = btc['55SMA'] = btc['close'].rolling(55).mean()
SMA_200 = btc['200SMA'] = btc['close'].rolling(200).mean()
EMA_200 = btc['200EMA'] = btc['close'].ewm(200).mean()
btc = btc.reset_index()
btc = btc.rename(columns={'time':'dtime'})
# btc.set_index(['dtime'], inplace=True)
btc_api = btc.drop(['close'], axis=1)
# btc_api

In [2771]:
# Read Full_Moon CSV, Clean index, infer Datetime
full_moon = pd.read_csv(csvpath1)
full_moon = full_moon.rename(columns = {' Date':'dtime'})
full_moon['dtime'] = full_moon['dtime']+' '+full_moon[' Time']
full_moon['Phase'] = full_moon.loc['dtime',:] = 1
full_moon.drop([' Time'], axis=1, inplace=True)
full_moon.drop(full_moon.index[-1], inplace=True)

full_moon['dtime'] = pd.to_datetime(full_moon['dtime'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
full_moon['dtime'] = pd.DataFrame(full_moon['dtime'] + timedelta(hours=-2))

full_moon

,dtime,Phase
0,2017-11-04 04:22:55,1.0
1,2017-12-03 14:46:59,1.0
2,2018-01-02 01:24:05,1.0
3,2018-01-31 12:26:44,1.0
4,2018-03-01 23:51:21,1.0
...,...,...
406,2050-09-01 08:30:54,1.0
407,2050-09-30 16:31:48,1.0
408,2050-10-30 02:16:00,1.0
409,2050-11-28 14:09:48,1.0


In [2772]:
# Timedelta + 15 days to create New_Moon Dataframe
new_moon = pd.DataFrame(full_moon['dtime'] + timedelta(days=14, hours=19, minutes=26))
new_moon['Phase'] = new_moon.loc['dtime',:] = -1
new_moon.drop(new_moon.index[-1], inplace=True)
new_moon['dtime'] = pd.to_datetime(new_moon['dtime'])

new_moon

,dtime,Phase
0,2017-11-18 23:48:55,-1.0
1,2017-12-18 10:12:59,-1.0
2,2018-01-16 20:50:05,-1.0
3,2018-02-15 07:52:44,-1.0
4,2018-03-16 19:17:21,-1.0
...,...,...
406,2050-09-16 03:56:54,-1.0
407,2050-10-15 11:57:48,-1.0
408,2050-11-13 21:42:00,-1.0
409,2050-12-13 09:35:48,-1.0


In [2773]:
full_moon['dtime'] = pd.to_datetime(full_moon['dtime']).dt.date
new_moon['dtime'] = pd.to_datetime(new_moon['dtime']).dt.date

In [2774]:
# Append Lunar Dataframes to create one table sorted by 'dtime'
phase_data = full_moon.append(new_moon)
phase_data.sort_values('dtime', inplace= True)
phase_data.reset_index(inplace=True)
phase_data.drop(['index'], axis = 1, inplace=True)
phase_data['dtime'] = pd.to_datetime(phase_data['dtime'])
# phase_data.set_index(['dtime'], inplace=True)
phase_data

,dtime,Phase
0,2017-11-04,1.0
1,2017-11-18,-1.0
2,2017-12-03,1.0
3,2017-12-18,-1.0
4,2018-01-02,1.0
...,...,...
817,2050-11-13,-1.0
818,2050-11-28,1.0
819,2050-12-13,-1.0
820,2050-12-28,1.0


In [2775]:
eclipse_df1 = pd.read_csv(lunar_eclipse, parse_dates=True, infer_datetime_format=True)
eclipse_df1.rename(columns={'Date':'dtime'}, inplace=True)
eclipse_df1['Eclipse'] = 1
eclipse_df1['dtime'] = pd.to_datetime(eclipse_df1['dtime'])
# eclipse_df1

In [2776]:
eclipse_df2 = pd.read_csv(solar_eclipse, parse_dates=True, infer_datetime_format=True)
eclipse_df2.rename(columns={'Date':'dtime'}, inplace=True)
eclipse_df2['Eclipse'] = -1
eclipse_df2['dtime'] = pd.to_datetime(eclipse_df2['dtime'])
eclipse_df2.drop(['Unnamed: 1'], axis = 1, inplace=True)
# eclipse_df2

In [2777]:
eclipse_data = eclipse_df1.append(eclipse_df2)
eclipse_data.sort_values('dtime', inplace= True)
eclipse_data.reset_index(inplace=True)
eclipse_data.drop(['index'], axis = 1, inplace=True)
eclipse_data['dtime'] = pd.to_datetime(eclipse_data['dtime'])
eclipse_data.columns
eclipse_data.set_index(['dtime'], inplace=True)
eclipse_data.tail(365)

,Eclipse
dtime,
2020-11-30,1
2020-12-14,-1
2021-05-26,1
2021-06-10,-1
2021-11-19,1
...,...
2099-09-29,1
2100-02-24,1
2100-03-10,-1


# Merge Dataframes on 'dtime'

In [2778]:

moon_merge = phase_data.merge(btc_api, on='dtime', how='outer')
moon_merge['dtime'] = pd.to_datetime(moon_merge['dtime'])
# moon_merge.set_index(['dtime'], inplace=True)
# moon_merge['Phase'] = moon_merge['Phase'].fillna(0)
# moon_merge.dropna()
moon_merge

,dtime,Phase,Price,8SMA,55SMA,200SMA,200EMA
0,2017-11-04,1.0,7363.80,6591.02000,4947.186545,NaN,4244.562399
1,2017-11-18,-1.0,7780.91,6994.40750,5772.459636,NaN,4614.534559
2,2017-12-03,1.0,11246.21,10220.33625,7131.205455,NaN,5235.694267
3,2017-12-18,-1.0,18972.32,17694.93375,9989.369636,NaN,6595.186019
4,2018-01-02,1.0,14754.13,14318.17500,12186.421273,NaN,7548.320014
...,...,...,...,...,...,...,...
2528,2022-06-24,NaN,21219.46,20433.01750,28900.205455,38802.50450,39023.116115
2529,2022-06-25,NaN,21474.19,20563.25875,28590.999273,38656.71040,38935.798199
2530,2022-06-26,NaN,21031.85,20822.95875,28273.160000,38509.30445,38846.713855
2531,2022-06-27,NaN,20718.16,20843.54375,27963.937818,38374.92600,38756.511995


In [2779]:
def Tru_Moo(x, y):
    if x - y > 0:
        return 1
    return -1

moon_data['Tru_Moo'] = moon_data.apply(lambda row: Tru_Moo(row['New Moon Price'], row['Full Moon Price']), axis=1)
moon_data['PCT Change'] = (moon_data['New Moon Price'] - moon_data['Full Moon Price']) / moon_data['New Moon Price']

#for ind in (moon_data.index):
#print(moon_data.iloc[:,2])
#print(moon_data.iloc[1:,3]- moon_data.iloc[:,2])

moon_data = moon_data.rename(columns={'New_Moon':'dtime'})
moon_data = moon_data.append(phase_data)
moon_data.drop(['New Moon Price','Full_Moon','Full Moon Price','Phase'], axis=1, inplace=True)
moon_data.sort_values('dtime', inplace= True)
moon_data

,dtime,Tru_Moo,PCT Change
0,2017-11-04,NaN,NaN
1,2017-11-18,NaN,NaN
0,2017-11-19,1.0,0.084405
2,2017-12-03,NaN,NaN
1,2017-12-18,1.0,0.407231
...,...,...,...
817,2050-11-13,NaN,NaN
818,2050-11-28,NaN,NaN
819,2050-12-13,NaN,NaN
820,2050-12-28,NaN,NaN


### Machine Learning

In [2780]:
eclipse_cat = moon_merge.merge(eclipse_data, on='dtime', how='outer')
eclipse_cat['Eclipse'] = eclipse_cat['Eclipse'].fillna(0)
# eclipse_cat['200SMA'] = eclipse_cat['200SMA'].fillna(0)
# eclipse_cat.set_index(['dtime'], inplace=True)
# eclipse_cat

In [2781]:
eclipse_cat = moon_data.merge(eclipse_cat, on='dtime', how='inner')
eclipse_cat['Tru_Moo'] = eclipse_cat['Tru_Moo'].fillna(0)
eclipse_cat['Phase'] = eclipse_cat['Phase'].fillna(0)
eclipse_cat['PCT Change'] = eclipse_cat['PCT Change'].fillna(0)
eclipse_cat.set_index(['dtime'],inplace=True)
eclipse_cat.dropna()

,Tru_Moo,PCT Change,Phase,Price,8SMA,55SMA,200SMA,200EMA,Eclipse
dtime,,,,,,,,,
2018-01-16,0.0,0.000000,-1.0,11282.49,13666.68250,14054.222545,7098.68290,8299.459816,0.0
2018-01-17,-1.0,-0.321735,0.0,11162.70,13253.45750,14111.482364,7142.37335,8321.833660,0.0
2018-01-31,0.0,0.000000,1.0,10226.86,11063.07625,14195.108000,7757.40785,8617.727548,1.0
2018-02-15,-1.0,-0.019246,-1.0,10033.75,8823.34625,11967.468364,8204.82945,8611.324876,-1.0
2018-02-15,0.0,0.000000,-1.0,10033.75,8823.34625,11967.468364,8204.82945,8611.324876,-1.0
...,...,...,...,...,...,...,...,...,...
2022-05-01,-1.0,-0.049688,-1.0,38480.53,38968.48000,41745.827091,47271.14055,42306.747639,0.0
2022-05-16,0.0,0.000000,1.0,29838.50,29931.58750,39784.711455,45151.20030,41642.369157,1.0
2022-05-30,0.0,0.000000,-1.0,31716.41,29522.77625,35674.930909,42802.09040,40830.124631,0.0


In [2782]:
eclipse_cat["signal"] = 0.0
#Buy Signal
eclipse_cat.loc[(eclipse_cat['Phase'] == 1), "signal"] = 1
#Sell Signal
eclipse_cat.loc[(eclipse_cat['Phase'] == -1) & (eclipse_cat['PCT Change'] > 0), "signal"] = -1

If New Moon is < Previous Full Moon = Do not sell.
Do not buy on full moon before eclipse.
When the 8 SMA > 200 SMA > 200 EMA = Bearish Cross, do not buy on full moon.
When the 8 SMA < 200 SMA > 200 EMA = Buy the full moon

In [2783]:
test_df = eclipse_cat.shift().copy().dropna()

In [2784]:
X = test_df[['Price','8SMA','55SMA','200SMA','200EMA','Phase','Tru_Moo','Eclipse']]

display(X.head())
display(X.tail())

,Price,8SMA,55SMA,200SMA,200EMA,Phase,Tru_Moo,Eclipse
dtime,,,,,,,,
2018-01-17,11282.49,13666.68250,14054.222545,7098.68290,8299.459816,-1.0,0.0,0.0
2018-01-31,11162.70,13253.45750,14111.482364,7142.37335,8321.833660,0.0,-1.0,0.0
2018-02-15,10226.86,11063.07625,14195.108000,7757.40785,8617.727548,1.0,0.0,1.0
2018-02-15,10033.75,8823.34625,11967.468364,8204.82945,8611.324876,-1.0,-1.0,-1.0
2018-03-01,10033.75,8823.34625,11967.468364,8204.82945,8611.324876,-1.0,0.0,-1.0


,Price,8SMA,55SMA,200SMA,200EMA,Phase,Tru_Moo,Eclipse
dtime,,,,,,,,
2022-05-16,38480.53,38968.48000,41745.827091,47271.14055,42306.747639,-1.0,-1.0,0.0
2022-05-30,29838.50,29931.58750,39784.711455,45151.20030,41642.369157,1.0,0.0,1.0
2022-05-31,31716.41,29522.77625,35674.930909,42802.09040,40830.124631,-1.0,0.0,0.0
2022-06-14,31782.16,29861.08625,35467.737818,42640.21970,40785.104143,0.0,1.0,0.0
2022-06-29,22118.37,27502.26750,32291.415818,40488.62110,39963.374566,1.0,0.0,0.0


In [2785]:
y = test_df["signal"]
y

dtime
2018-01-17    0.0
2018-01-31    0.0
2018-02-15    1.0
2018-02-15    0.0
2018-03-01    0.0
             ... 
2022-05-16    0.0
2022-05-30    1.0
2022-05-31    0.0
2022-06-14    0.0
2022-06-29    1.0
Name: signal, Length: 165, dtype: float64

In [2786]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2018-01-17 00:00:00


In [2787]:
# Select the ending period for the training data with an offset of 18 months
training_end = X.index.min()+ DateOffset(years=3)

# Display the training end date
print(training_end)

2021-01-17 00:00:00


In [2788]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [2789]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [2790]:
from sklearn.preprocessing import StandardScaler

In [2791]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [2792]:
X_train_scaled.shape

(112, 8)

In [2793]:
from sklearn import svm
from sklearn.metrics import classification_report

In [2794]:
svm_model = svm.SVC()
 
# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

array([ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1., -1.])

In [2795]:
training_report = classification_report(y_train, training_signal_predictions)

# Display the report
print(training_report)

              precision    recall  f1-score   support

        -1.0       1.00      1.00      1.00        13
         0.0       1.00      1.00      1.00        62
         1.0       1.00      1.00      1.00        37

    accuracy                           1.00       112
   macro avg       1.00      1.00      1.00       112
weighted avg       1.00      1.00      1.00       112



In [2796]:
testing_signal_predictions = svm_model.predict(X_test_scaled)
testing_signal_predictions.shape

(53,)

In [2797]:
# testing_signal_predictions.shape
X_test_scaled.shape
# y_test.shape 

(53, 8)

In [2798]:
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         5
         0.0       0.57      1.00      0.72        30
         1.0       0.00      0.00      0.00        18

    accuracy                           0.57        53
   macro avg       0.19      0.33      0.24        53
weighted avg       0.32      0.57      0.41        53



/Users/JDylan/opt/anaconda3/envs/machnlearning/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/JDylan/opt/anaconda3/envs/machnlearning/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/JDylan/opt/anaconda3/envs/machnlearning/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

In [2799]:
# Create a New Temporary Data Frame to Finalize the Moon Price Behavior Data

frame = {'Full Moon': moon_data.iloc[:,0], 'New Moon': moon_data.iloc[:,1], 'Full Moon Price': moon_data.iloc[:,2], 'New Moon Price': moon_data.iloc[:,3]} #'New Moon Price': moon_data.iloc[1:,3]}
tmpdf = pd.DataFrame(frame)
tmpdf['Difference']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'] #.shift(1)
tmpdf['Percentage Difference'] = ((tmpdf['Full Moon Price'] - tmpdf['New Moon Price']) / tmpdf['New Moon Price'])*100
tmpdf['Trade Profit']= tmpdf['Difference'].cumsum()
tmpdf['Buy and Hold']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'][0]

def returnBoolFM(x):
    if x < 0:
        return True
    return False

def returnBoolNM(x):
    if x > 0:
        return True
    return False

tmpdf['Did We Profit this Month'] = tmpdf.apply(lambda row: returnBoolNM(row['Difference']), axis = 1)


tmpdf

IndexError: single positional indexer is out-of-bounds

In [ ]:
#plot Difference column vs the buy and hold column
buy_hold_glyph = tmpdf.hvplot.line(
    x = 'Full Moon',
    y = 'Buy and Hold',
    xlabel = 'Date',
    ylabel = 'Buy and Hold',
    title = 'Moon Phases Vs. Buy and Hold',
    color = 'red',
    legend = True
)

moon_glyph = tmpdf.hvplot.line(
    x = 'Full Moon',
    y = 'Trade Profit',
    xlabel = 'Date',
    ylabel = 'Difference',
    title = 'Moon Phases Vs. Buy and Hold',
    color = 'blue',
    legend = True
    
)

moon_glyph*buy_hold_glyph

In [ ]:
# Define a function to count the Trues in the 'Did We Profit this Month' column
def count_tr(x):
    return x.sum()
number_of_Trues = (count_tr(tmpdf['Did We Profit this Month']))

#Count the number of rows in the Did We Profit this Month column

number_of_Rows = tmpdf['Did We Profit this Month'].count()

# The PERCENTAGE of times you would profit if you bought on Full Moon and sold on New Moon each month

print(f'{(number_of_Trues/number_of_Rows)*100:.2f}% is the percentage of times you would profit if you bought BTC on the full moon and sold it on the new moon of every month!')

In [ ]:
#7. create column for the all time high and low
# moon_data['All Time High'] = Price.max()
# moon_data


In [ ]:
fear_data = json.dumps(FearGreedUrl["data"][0], indent=4)
fear_json = json.loads(fear_data)
fear_json


In [ ]:
fear_df = pd.DataFrame(data=fear_json, columns=['value', 'value_classification', 'timestamp'])